1) Создать скрипт для анализа тестовых файлов - сбора статистики упоминания слов. Аргументом командной строки является имя файла, который нужно проанализировать. Второй аргумент - количество слов, которые нужно выводить на экран (если аргумент не указан, то выводим статистику по 100 словам, статистика выводится в порядке убывания частоты слов, по каждому слову на экран выводится строка в формате "<слово> <частота>"). Если второй аргумент не целое число, то его рассматриваем как имя файла, в который в формате CSV с заголовком сохраняем статистику слов (Первый столбец - слово, второй столбце - частота).


In [ ]:
import sys
import csv
import re
from collections import Counter
from google.colab import files
uploaded = files.upload()

def analyze_file(file_name, num_words=100):
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            text = file.read()
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")
        return

    # Remove special characters and split text into words
    words = re.findall(r'\b\w+\b', text.lower())

    # Count word frequency
    word_freq = Counter(words)

    # Sort word frequency in descending order
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

    if type(num_words) == int:
        # Display word frequency
        print(f"Top {num_words} words and their frequencies:")
        for word, freq in sorted_word_freq[:num_words]:
            print(f"{word}: {freq}")
    else:
        # Save word frequency to CSV file
        try:
            with open("new_csv", 'w', newline='', encoding='utf-8') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow(['Word', 'Frequency'])
                for word, freq in sorted_word_freq:
                    csv_writer.writerow([word, freq])
            print(f"Word frequency saved to '{num_words}'.")
        except:
            print("Unable to save word frequency to CSV file.")

if __name__ == '__main__':
    if len(sys.argv) < 2:
        print("Usage: python script.py <file_name> [num_words]")
    else:
        file_name = sys.argv[1]
        if len(sys.argv) == 3:
            num_words = sys.argv[2]
        else:
            num_words = 100
        analyze_file(file_name, num_words)


Saving 1.txt.txt to 1.txt (1).txt
File '-f' not found.


2) Создать скрипт, которому в командной строке передаются имена файлов. В файлах в формате CSV с заголовком содержится информация о частоте слов: первый столбец - слово, второй столбце - частота. Выполнить агрегацию информации о частоте слов из всех переданных файлов (т.е. рассчитать суммарную частоту для всех упомянутых слов). Результат сохранить в файл с именем, которое введет пользователь в ответ на соответствующий запрос.


In [ ]:
import sys
import csv
from google.colab import files
uploaded = files.upload()

# Считываем имена файлов из аргументов командной строки
files = sys.argv[1:]

# Создаем словарь для хранения частот слов
word_freq = {}

# Обходим все переданные файлы
for file_name in files:
    with open(file_name, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Пропускаем заголовок
        for row in reader:
            word = row[0]
            freq = int(row[1])
            if word in word_freq:
                word_freq[word] += freq
            else:
                word_freq[word] = freq

# Запрашиваем у пользователя имя файла для сохранения результата
output_file = input("Введите имя файла для сохранения результата: ")

# Записываем результаты в файл
with open(output_file, 'w') as result_file:
    writer = csv.writer(result_file)
    writer.writerow(['Слово', 'Суммарная частота'])
    for word, freq in word_freq.items():
        writer.writerow([word, freq])

print("Результат успешно сохранен в файл", output_file)

2.1) Скрипт должен адекватно реагировать на ситуацию отсутствия файлов и другие проблемы с вводом/выводом.

In [ ]:
from google.colab import files
uploaded = files.upload()
try:
    with open('file.txt', 'r') as file:
        data = file.read()
        print(data)
except FileNotFoundError:
    print("Файл не найден")
except IOError:
    print("Ошибка ввода/вывода")

2.2) Оформить модуль с функциями, которые используются в задании 1 и 2.

In [ ]:
# word_stats_module.py

import sys
import csv
from collections import Counter
from google.colab import files
uploaded = files.upload()

def count_word_frequency(file_name):
    words = []

    with open(file_name, 'r') as file:
        for line in file:
            words.extend(line.strip().split())

    word_freq = Counter(words)
    return word_freq

def display_word_stats(word_freq, num_words=100):
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

    if isinstance(num_words, int):
        num_words = min(num_words, len(sorted_word_freq))
        for i in range(num_words):
            print(f"{sorted_word_freq[i][0]} {sorted_word_freq[i][1]}")
    else:
        with open(num_words, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Word', 'Frequency'])
            for key, value in sorted_word_freq:
                writer.writerow([key, value])

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python script_name.py <file_name> [num_words]")
    else:
        file_name = sys.argv[1]
        if len(sys.argv) == 3:
            num_words = sys.argv[2] if sys.argv[2].isdigit() else int(sys.argv[2])
        else:
            num_words = 100

        word_freq = count_word_frequency(file_name)
        display_word_stats(word_freq, num_words)


Для выполнения второй задачи, можно создать дополнительный скрипт, который будет использовать функции из этого модуля:

In [ ]:
# aggregate_word_stats.py

import sys
import csv
from word_stats_module import count_word_frequency
from google.colab import files
uploaded = files.upload()

if __name__ == "__main__":
    if len(sys.argv) < 3:
        print("Usage: python aggregate_word_stats.py <output_file> <input_files>")
    else:
        output_file = sys.argv[1]
        input_files = sys.argv[2:]

        total_word_freq = Counter()

        for file_name in input_files:
            word_freq = count_word_frequency(file_name)
            total_word_freq += word_freq

        with open(output_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Word', 'Total Frequency'])
            for key, value in total_word_freq.items():
                writer.writerow([key, value])

3.1) Написать программу, которая запускается из консоли и считает сумму переданных в неё чисел

In [ ]:
import sys

def calculate_sum(numbers):
    return sum(numbers)

if __name__ == "__main__":
    numbers = [int(num) for num in sys.argv[1:]]  # Преобразуем аргументы в целые числа
    result = calculate_sum(numbers)
    print(f"Сумма чисел: {result}")

Вы можете запустить эту программу из командной строки следующим образом:

python имя_файла.py число1 число2 число3 ...

Например:

python sum.py 5 10 15

Программа выведет сумму чисел (5 + 10 + 15 = 30).

3.2) Добавить в эту программу произведение

In [ ]:
import sys

def calculate_sum(numbers):
    return sum(numbers)

def calculate_product(numbers):
    result = 1
    for num in numbers:
        result *= num
    return result

if __name__ == "__main__":
    numbers = [int(num) for num in sys.argv[1:]]  # Преобразуем аргументы в целые числа
    sum_result = calculate_sum(numbers)
    product_result = calculate_product(numbers)
    print(f"Сумма чисел: {sum_result}")
    print(f"Произведение чисел: {product_result}")

Вы можете запустить эту программу из командной строки следующим образом:

python имя_файла.py число1 число2 число3 ...

Например:

python sum_product.py 2 3 4

Программа выведет сумму чисел (2 + 3 + 4 = 9) и произведение чисел (2 * 3 * 4 = 24).


3.3) По переданному параметру считать либо сумму, либо произведение

In [ ]:
import sys

def calculate_sum(numbers):
    return sum(numbers)

def calculate_product(numbers):
    result = 1
    for num in numbers:
        result *= num
    return result

if __name__ == "__main__":
    args = sys.argv[1:]

    if len(args) < 3:
        print("Пожалуйста, укажите числа для вычисления и режим работы (сумма или произведение).")
    else:
        numbers = [int(num) for num in args[:-1]]  # Преобразуем числа в целые числа
        mode = args[-1].lower()  # Режим работы (сумма или произведение)

        if mode == "сумма":
            result = calculate_sum(numbers)
            print(f"Сумма чисел: {result}")
        elif mode == "произведение":
            result = calculate_product(numbers)
            print(f"Произведение чисел: {result}")
        else:
            print("Неверно указан режим работы. Укажите 'сумма' или 'произведение'.")

Пожалуйста, укажите числа для вычисления и режим работы (сумма или произведение).


Вы можете запустить эту программу из командной строки следующим образом:

python имя_файла.py число1 число2 ... сумма или python имя_файла.py число1 число2 ... произведение

Например:

python sum_product_mode.py 2 3 4 сумма

Программа выведет сумму чисел (2 + 3 + 4 = 9).

3.4) Оформить дополнительный модуль.

In [ ]:
import sys

def calculate_sum(numbers):
    return sum(numbers)

def calculate_product(numbers):
    result = 1
    for num in numbers:
        result *= num
    return result

if __name__ == "__main__":
    args = sys.argv[1:]

    if len(args) < 1:
        print("Пожалуйста, укажите числа для вычисления.")
    else:
        numbers = [int(num) for num in args]  # Преобразуем числа в целые числа

        sum_result = calculate_sum(numbers)
        print(f"Сумма чисел: {sum_result}")

        product_result = calculate_product(numbers)
        print(f"Произведение чисел: {product_result}")


In [ ]:
# Для выполнения задачи 3.3 и 3.4, вы можете создать дополнительный модуль calculator.py, который будет содержать функции для расчета суммы и произведения чисел:
def calculate_sum(numbers):
    return sum(numbers)

def calculate_product(numbers):
    result = 1
    for num in numbers:
        result *= num
    return result


Затем в основной программе вы можете импортировать эти функции и использовать их для вычисления суммы или произведения в зависимости от переданного параметра.

4) Реализовать модуль calculate.py, содержащий набор функций для работы с целыми числами, записанными как на русском языке (например: тридцать восемь), так и в виде простых арифметических выражений (например: 2 + 3).
На базе этого модуля подготовить скрипт calculate.py со следующими возможностями:
1) python claculate.py to_numbers my_file.txt
Вызов приводит к преобразованию всех чисел, написанных на русском языке в числовом виде и сохранению изменений в файл с тем же именем.
python claculate.py to_numbers my_file.txt my_file_res.txt
(результат сохраняется в файл с именем my_file_res.txt)


In [ ]:
from num2words import num2words
import re
from google.colab import files
uploaded = files.upload()

def convert_to_numbers(text):
    def replace_number(match):
        number_text = match.group(0)
        try:
            number = int(number_text)
            return str(number)
        except ValueError:
            try:
                number = num2words(int(match.group(0)), lang='ru')
                return number
            except:
                return number_text

    pattern = r'\b(?:ноль|один|два|три|четыре|пять|шесть|семь|восемь|девять|десять|\
        одиннадцать|двенадцать|тринадцать|четырнадцать|пятнадцать|шестнадцать|семнадцать|восемнадцать|девятнадцать|\
        двадцать|тридцать|сорок|пятьдесят|шестьдесят|семьдесят|восемьдесят|девяносто|сто|тысяча)\b'

    return re.sub(pattern, replace_number, text)

def process_file(input_file, output_file=None):
    with open(input_file, 'r', encoding='utf-8') as file:
        content = file.read()

    converted_content = convert_to_numbers(content)

    if output_file:
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(converted_content)
    else:
        with open(input_file, 'w', encoding='utf-8') as file:
            file.write(converted_content)

if __name__ == "__main__":
    import sys

    if len(sys.argv) < 3:
        print("Пожалуйста, укажите режим работы (to_numbers) и имя файла.")
    elif sys.argv[1] == "to_numbers":
        input_file = sys.argv[2]
        output_file = sys.argv[3] if len(sys.argv) > 3 else None
        process_file(input_file, output_file)


Этот модуль использует библиотеку num2words для преобразования чисел на русском языке в числовой формат. Функция convert_to_numbers заменяет числа на русском языке в тексте на соответствующие числа в числовом формате. Функция process_file читает содержимое файла, преобразует числа и сохраняет их обратно в тот же файл или в новый файл.

Вы можете использовать этот скрипт следующим образом:

python calculate.py to_numbers my_file.txt my_file_res.txt

Это приведет к преобразованию чисел на русском языке в файле my_file.txt и сохранению результатов в файле my_file_res.txt.